In [1]:
import awkward1 as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import numpy as np
from tqdm import tqdm,trange
import matplotlib.pyplot as plt
import numba

In [2]:
infile = "data/signal/wza_UL18.root" # --signal
#infile='data/data/7B56E217-555E-1C41-9494-491849A9835F_skim_2ElIdPt20.root' # --data 
#infile = 'data/data/CA42F3A2-614F-4A4F-AF18-F6E66CDA2C85_skim_2ElIdPt20.root'
#infile = 'data/Ntuple/EGamma_Run2018B.root'
#infile="data/mc/59AB328B-F0E3-F544-98BB-E5E55577C649_skim_2ElIdPt20.root" # --mc


year='2018'

In [3]:
events = NanoEventsFactory.from_root(infile, schemaclass=NanoAODSchema).events()

In [4]:
# Trigger set
doubleelectron_triggers  ={
    '2018': [
            "Ele23_Ele12_CaloIdL_TrackIdL_IsoVL", # Recomended
            ]
}



singleelectron_triggers = { #2017 and 2018 from monojet, applying dedicated trigger weights
        '2016': [
            'Ele27_WPTight_Gsf',
            'Ele105_CaloIdVT_GsfTrkIdT'
        ],
        '2017': [
            'Ele35_WPTight_Gsf',
            'Ele115_CaloIdVT_GsfTrkIdT',
            'Photon200'
        ],
        '2018': [
            'Ele32_WPTight_Gsf',    # Recomended
        ]
    }

In [5]:
# << flat dim helper function >>
def flat_dim(arr):

#   if len(ak.to_numpy(arr).shape) > 1:
#       sub_arr = ak.flatten(arr)
#   else:
#       sub_arr = arr
    sub_arr = ak.flatten(arr)
    mask = ~ak.is_none(sub_arr)

    return ak.to_numpy(sub_arr[mask])
# << drop na helper function >>
def drop_na(arr):

    mask = ~ak.is_none(arr)

    return arr[mask]
# << drop na helper function >>
def drop_na_np(arr):

    mask = ~np.isnan(arr)

    return arr[mask]
# << Sort by PT  helper function >>
def sort_by_pt(ele,pho,jet):
    ele = ele[ak.argsort(ele.pt,ascending=False,axis=1)]
    pho = pho[ak.argsort(pho.pt,ascending=False,axis=1)]
    jet = jet[ak.argsort(jet.pt,ascending=False,axis=1)]

    return ele,pho,jet

# << Delta R helper function >>
def delta_r(eta1, eta2, phi1, phi2):
    deta = abs(eta1 - eta2)
    dphi = abs(np.mod(phi1 - phi2 + np.pi, 2*np.pi) - np.pi)
    dr = np.sqrt(deta**2 + dphi**2)
    return deta, dphi, dr

In [6]:
# double lepton trigger
is_double_ele_trigger=True
if not is_double_ele_trigger:
    double_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    double_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in doubleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        double_ele_triggers_arr = double_ele_triggers_arr | events.HLT[path]


# single lepton trigger
is_single_ele_trigger=True
if not is_single_ele_trigger:
    single_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    single_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in singleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        single_ele_triggers_arr = single_ele_triggers_arr | events.HLT[path]

# Sort particle order by PT  # RunD --> has problem
events.Electron,events.Photon,events.Jet = sort_by_pt(events.Electron,events.Photon,events.Jet)

Initial_events = events
#events = events[single_ele_triggers_arr | double_ele_triggers_arr]
events = events[double_ele_triggers_arr]

cut1 = np.ones(len(events))
# Particle Identification


In [7]:
'''
class particle_set():
    
    def __init__(self,events_,isMu):
        self.events_ = events_
        self.Electron = events_.Electron
        self.isMu = isMu
        
        if self.isMu: self.Muon = events_.Muon;
        
        self.Photon = events_.Photon
        self.Jet = events_.Jet
        self.MET = events_.MET
        
    def evt_selection(self,events,event_mask):
        self.Electron = self.Electron[event_mask]
        self.Photon = self.Photon[event_mask]
        self.Jet = self.Jet[event_mask]

        if self.isMu:
            self.Muon = self.Muon
            return self.Electron,self.Muon,self.Photon,self.Jet,self.MET
        else:
            return self.Electron,self.Photon,self.Jet,self.MET
'''

'\nclass particle_set():\n    \n    def __init__(self,events_,isMu):\n        self.events_ = events_\n        self.Electron = events_.Electron\n        self.isMu = isMu\n        \n        if self.isMu: self.Muon = events_.Muon;\n        \n        self.Photon = events_.Photon\n        self.Jet = events_.Jet\n        self.MET = events_.MET\n        \n    def evt_selection(self,events,event_mask):\n        self.Electron = self.Electron[event_mask]\n        self.Photon = self.Photon[event_mask]\n        self.Jet = self.Jet[event_mask]\n\n        if self.isMu:\n            self.Muon = self.Muon\n            return self.Electron,self.Muon,self.Photon,self.Jet,self.MET\n        else:\n            return self.Electron,self.Photon,self.Jet,self.MET\n'

In [8]:
Electron = events.Electron
Photon = events.Photon
Jet = events.Jet
MET = events.MET
Muon = events.Muon

In [9]:
isData = 'genWeight' not in events.fields

In [10]:
def Particle_selection(ele,pho,mu):
    
    # Electron selection
    EleSelmask = EleSelmask = ((ele.pt > 10) & (np.abs(ele.eta + ele.deltaEtaSC) < 1.4442) & (ele.cutBased > 2) & (abs(ele.dxy) < 0.05) & (abs(ele.dz) < 0.1)) | \
        ((ele.pt > 10) & (np.abs(ele.eta + ele.deltaEtaSC) > 1.5660) & (np.abs(ele.eta + ele.deltaEtaSC) < 2.5) & (ele.cutBased > 2) & (abs(ele.dxy) < 0.1) & (abs(ele.dz) < 0.2))


    # Photon selection
    isgap_mask = (abs(pho.eta) < 1.442)  |  ((abs(pho.eta) > 1.566) & (abs(pho.eta) < 2.5))
    Pixel_seed_mask = ~pho.pixelSeed
    
    
    if not isData:
        isPrompt = (Photon.genPartFlav == 1) | (Photon.genPartFlav == 11)
        PhoSelmask = (pho.pt > 20) & (pho.cutBased > 1) & isgap_mask &  Pixel_seed_mask & isPrompt
    if isData:
        PhoSelmask = (pho.pt > 20) & (pho.cutBased > 1) & isgap_mask &  Pixel_seed_mask     
    
    # Muon only used to calculate dR
    MuSelmask = (mu.pt > 10) & (abs(mu.eta) < 2.5)  & (mu.tightId) & (mu.pfRelIso04_all < 0.15)
    

    return EleSelmask,PhoSelmask,MuSelmask

In [11]:
Electron_mask, Photon_mask, Muon_mask  = Particle_selection(Electron,Photon,Muon)

In [12]:
Electron = Electron[Electron_mask]
Photon   = Photon[Photon_mask]
Muon = Muon[Muon_mask] # for Photon dR

In [13]:
# Event Selectiom
Ele_channel_mask = ak.num(Electron)  == 3
Pho_channel_mask = ak.num(Photon) > 0


Electron = Electron[Ele_channel_mask & Pho_channel_mask ]
Photon   = Photon[Ele_channel_mask & Pho_channel_mask]
Jet = Jet[Ele_channel_mask & Pho_channel_mask]
Muon = Muon[Ele_channel_mask & Pho_channel_mask]
MET = MET[Ele_channel_mask & Pho_channel_mask]

In [14]:
print("Events passing triggers and skiimig: ",len(Photon))

Events passing triggers and skiimig:  325


In [15]:
#@numba.njit
def make_dR_mask(evt_electron, evt_photon, evt_muon, builder):
    
    
    for evt_idx in range(len(evt_photon)): # EvtLoop
        builder.begin_list()
        Pho = evt_photon[evt_idx]
        Ele = evt_electron[evt_idx]
        Mu  = evt_muon[evt_idx]
        
        for pho_idx in range(len(Pho)): # PhoLoop


            is_pass_dR = True
            for ele_idx in range(len(Ele)): #
                if Pho[pho_idx].delta_r(Ele[ele_idx]) < 0.5 : is_pass_dR = False
                
                    
            if len(Mu) != 0:
                for mu_idx in range(len(Mu)):
                    if Pho[pho_idx].delta_r(Mu[mu_idx]) < 0.5 : is_pass_dR = False
                
            #print(is_pass_dR)
            builder.boolean(is_pass_dR)
        builder.end_list()
        
    return builder

In [16]:
pho_dR_mask = make_dR_mask(Electron,Photon,Muon,ak.ArrayBuilder()).snapshot()

In [17]:
Photon = Photon[pho_dR_mask]

In [18]:
Pho_evtselt_mask = ak.num(Photon) > 0

Electron = Electron[Pho_evtselt_mask]
Photon   = Photon[Pho_evtselt_mask]
Jet = Jet[Pho_evtselt_mask]
Muon = Muon[Pho_evtselt_mask]
MET = MET[Pho_evtselt_mask]


print("Paritcle selection: ",len(Photon))

Paritcle selection:  236


In [19]:
Electron_mask,Photon_mask, _ = Particle_selection(Electron,Photon,Muon)
Electron = Electron[Electron_mask]
Photon  = Photon[Photon_mask]

In [20]:
##-----------  Cut flow3: Electron Selection --> OSSF 
# OSSF index maker
@numba.njit
def find_3lep(events_leptons,builder):
    for leptons in events_leptons:

        builder.begin_list()
        nlep = len(leptons)
        for i0 in range(nlep):
            for i1 in range(i0+1,nlep):
                if leptons[i0].charge + leptons[i1].charge != 0: continue;

                for i2 in range(nlep):
                    if len({i0,i1,i2}) < 3: continue;
                    builder.begin_tuple(3)
                    builder.index(0).integer(i0)
                    builder.index(1).integer(i1)
                    builder.index(2).integer(i2)
                    builder.end_tuple()
        builder.end_list()
    return builder

eee_triplet_idx = find_3lep(Electron,ak.ArrayBuilder()).snapshot()

# OSSF cut
ossf_mask = ak.num(eee_triplet_idx) == 2
eee_triplet_idx = eee_triplet_idx[ossf_mask]

Electron= Electron[ossf_mask]
Photon= Photon[ossf_mask]
Jet= Jet[ossf_mask]
MET = MET[ossf_mask]

In [21]:
Triple_electron = [Electron[eee_triplet_idx[idx]] for idx in "012"]
from coffea.nanoevents.methods import vector
ak.behavior.update(vector.behavior)

def TLorentz_vector(vec):
    vec = ak.zip(
    {
                "x":vec.x,
                "y":vec.y,
                "z":vec.z,
                "t":vec.t
    },
    with_name = "LorentzVector"
    )
    return vec

def TLorentz_vector_cylinder(vec):

    vec = ak.zip(
    {
         "pt": vec.pt,
         "eta": vec.eta,
         "phi": vec.phi,
         "mass": vec.mass,
    },
    with_name="PtEtaPhiMLorentzVector",
    )

    return vec



Triple_eee = ak.zip({"lep1":Triple_electron[0],
                            "lep2":Triple_electron[1],
                         "lep3":Triple_electron[2],
                         "p4":TLorentz_vector(Triple_electron[0]+Triple_electron[1])})


In [22]:
bestZ_idx = ak.singletons(ak.argmin(abs(Triple_eee.p4.mass - 91.1876), axis=1))
Triple_eee = Triple_eee[bestZ_idx]
leading_ele, subleading_ele, Third_ele = ak.flatten(TLorentz_vector_cylinder(Triple_eee.lep1)),ak.flatten(TLorentz_vector_cylinder(Triple_eee.lep2)),ak.flatten(TLorentz_vector_cylinder(Triple_eee.lep3))

In [23]:
def make_leading_pair(target,base):
    return target[ak.argmax(base.pt,axis=1,keepdims=True)]

leading_pho     = make_leading_pair(Photon,Photon)

In [24]:
print("OSSF selection: ",len(Triple_eee))

OSSF selection:  235


In [25]:
# bjet veto
bJet_selmask = (Jet.btagCMVA > -0.5844)
bJet_veto    = ak.num(Jet[bJet_selmask])==0
cut5 = np.ones(ak.sum(ak.num(leading_pho[bJet_veto] > 0 ))) * 5


# Z mass window
diele             = Triple_eee.p4
zmass_window_mask = ak.firsts((diele.mass) > 60 | (diele.mass < 120)) # signal region
#zmass_window_mask = ak.firsts(diele.mass) > 4  # control region


# M(eea) cuts 
eeg_vec           = diele + leading_pho
Meeg_mask         = ak.firsts(eeg_vec.mass > 120)

# Electron PT cuts
Elept_mask = ak.firsts((Triple_eee.lep1.pt > 25) & (Triple_eee.lep2.pt > 10) & (Triple_eee.lep3.pt > 25))

# MET cuts
MET_mask = MET.pt > 20

# Mask
Event_sel_mask   = bJet_veto & zmass_window_mask & Meeg_mask & Elept_mask & MET_mask # SR
#Event_sel_mask  = bJet_veto & zmass_window_mask & Meeg_mask & Elept_mask & MET_mask  & Loose_muon_veto_mask # SR (beta)
#Event_sel_mask  = bJet_veto & zmass_window_mask & Elept_mask & MET_mask   # CL


# Apply cyts
Triple_eee_sel   = Triple_eee[Event_sel_mask]
leading_pho_sel   = leading_pho[Event_sel_mask]
# Photon  EE and EB
isEE_mask = leading_pho.isScEtaEE
isEB_mask = leading_pho.isScEtaEB
Pho_EE = leading_pho[isEE_mask & Event_sel_mask]
Pho_EB = leading_pho[isEB_mask & Event_sel_mask]
MET_sel           = MET[Event_sel_mask]

In [26]:
print("Event selection: ",len(Triple_eee_sel))

Event selection:  53


In [27]:
leading_ele, subleading_ele, Third_ele = ak.flatten(TLorentz_vector_cylinder(Triple_eee_sel.lep1)),ak.flatten(TLorentz_vector_cylinder(Triple_eee_sel.lep2)),ak.flatten(TLorentz_vector_cylinder(Triple_eee_sel.lep3))